# Prepare dataset

### Dowload dataset

In [ ]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
# !wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip

--2022-11-27 09:22:41--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  1.90MB/s    in 2.0s    

2022-11-27 09:22:45 (1.90 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]



### Download file stopword

In [ ]:
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt

--2022-11-27 09:22:45--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords-dash.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0s      

2022-11-27 09:22:45 (132 MB/s) - ‘vietnamese-stopwords-dash.txt’ saved [20475/20475]



### Unzip dataset

In [ ]:
!unzip -q /content/e2eqa-train+public_test-v1.zip
# !unzip -q /content/wikipedia_20220620_cleaned.zip

# Install and import package

In [ ]:
!pip install pyvi
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 13.3 MB/s 
     |████████████████████████████████| 965 kB 49.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 10.9 MB/s 
     |████████████████████████████████| 182 kB 76.9 MB/s 
     |████████████████████████████████| 7.6 MB 85.9 MB/s 


In [ ]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# Read dataset and stopword file

In [ ]:
data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
data = json.load(data_file)

In [ ]:
stopword_file = open('vietnamese-stopwords-dash.txt', 'r')
stopword = stopword_file.read()

In [ ]:
stopword = stopword.split('\n')

# Postprocessing Output

In [ ]:
def is_date_format(s):
    x = s.lower()
    return ('ngày' in x) or ('tháng' in x) or ('năm' in x)

In [ ]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [ ]:
import re
def to_date_format(s):
    x = re.findall(r'\d+', s)
    if len(x)==1:
        return f'năm {x[0]}'
    elif len(x)==2:
        return f'tháng {x[0]} năm {x[1]}'
    elif len(x)==3:
        return f'ngày {x[0]} tháng {x[1]} năm {x[2]}'
    else:
        return ''

In [ ]:
def to_number_format(s):
    s = s.translate((str.maketrans('', '', string.punctuation)))
    s = s.strip()
    s = " ".join(s.split())
    return s

In [ ]:
import string 
def to_wiki_format(s):
    s = s.translate((str.maketrans('', '', string.punctuation)))
    s = s.strip()
    s = " ".join(s.split())
    s = s.replace(' ', '_')
    return 'wiki/' + s

#QA_model

In [ ]:
def remove_stopword(lsword):
  return lsword
  res = []
  for word in lsword.split():
    if not word in stopword:
      res.append(word) 
  return res

In [ ]:
questions_file = open('e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json')
questions = json.load(questions_file)
# questions

# TF_IDF

In [ ]:
# !pip install pandarallel
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 11.4 MB/s 
     |████████████████████████████████| 235 kB 62.7 MB/s 
     |████████████████████████████████| 591 kB 68.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import underthesea
import json
import os
from tqdm.autonotebook import tqdm
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from underthesea import text_normalize
from sklearn.decomposition import TruncatedSVD
import math
import torch

In [ ]:
def combine_metadata_wiki(item):
    newItem = {}
    # newItem['title'] = item['title']
    newItem['text'] = item['text']
    return newItem
def count_word_in_dataset_wiki(items):
    '''
    count number of word in dataset
    '''
    nameCounter = {}
    descCounter = {}
    for item in items:
        # for word in item['unigram_title'].keys():
        #     if word in nameCounter.keys():
        #         nameCounter[word] += 1
        #     else:
        #         nameCounter[word] = 1
        for word in item['unigram_text'].keys():
            if word in descCounter.keys():
                descCounter[word] += 1
            else:
                descCounter[word] = 1
    return  descCounter

def count_unigram(text):
    '''
    Count appearance number for each vocabulary
    '''
    counter = {}
    words = str(text).split()
    vocabs = set(words)
    for vocab in vocabs:
        if not vocab.isdigit():
            counter[vocab] = words.count(vocab)
    return counter


In [ ]:
def combine_metadata_wiki_question(item):
    newItem = {}
    # newItem['title'] = item['title']
    newItem['question'] = item['question']
    return newItem
def count_word_in_dataset_wiki_question(items):
    '''
    count number of word in dataset
    '''
    nameCounter = {}
    descCounter = {}
    for item in items:
        # for word in item['unigram_title'].keys():
        #     if word in nameCounter.keys():
        #         nameCounter[word] += 1
        #     else:
        #         nameCounter[word] = 1
        for word in item['unigram_question'].keys():
            if word in descCounter.keys():
                descCounter[word] += 1
            else:
                descCounter[word] = 1
    return  descCounter

def count_unigram(text):
    '''
    Count appearance number for each vocabulary
    '''
    counter = {}
    words = str(text).split()
    vocabs = set(words)
    for vocab in vocabs:
        if not vocab.isdigit():
            counter[vocab] = words.count(vocab)
    return counter


In [ ]:
for qu in questions['data']:
  # qu['title'] = qu['question']
  # qu['text'] = qu.pop('question')
  qu['question'] = remove_stopword(qu['question'])
  qu['text'] = qu['question']
for da in data['data']:
  da['text'] = remove_stopword(da['text'])
  da['question'] = remove_stopword(da['question'] )
items=data['data']+questions['data']

In [ ]:
newItems_wiki_question = []
for item in tqdm(items):
    newItem = combine_metadata_wiki_question(item)
    # newItem['unigram_title'] = count_unigram(newItem['title'])
    newItem['unigram_question'] = count_unigram(newItem['question'])
    newItems_wiki_question.append(newItem)

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
newItems_wiki = []
for item in tqdm(items):
    newItem = combine_metadata_wiki(item)
    # newItem['unigram_title'] = count_unigram(newItem['title'])
    newItem['unigram_text'] = count_unigram(newItem['text'])
    newItems_wiki.append(newItem)

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
avgdl = sum(len(sentence['text']) for sentence in data['data']) / len(data['data'])
def bm25(doc_len, corpus_len, doc_counter, corpus_counter, k=2,b=0.75):
    vector_len = len(corpus_counter)
    tfidf_vector = np.zeros((vector_len,))
    for i, key in enumerate(corpus_counter.keys()):
        if key in doc_counter.keys():
            tf = (k+1)*doc_counter[key]/(doc_counter[key]+k*(1-b+b*doc_len/avgdl) )
            idf = math.log((corpus_len-corpus_counter[key]+0.5)/(corpus_counter[key]+0.5)+1)
            tfidf_vector[i] = tf*idf
    return tfidf_vector

In [ ]:
textCounter = count_word_in_dataset_wiki(newItems_wiki)
textCounter_question = count_word_in_dataset_wiki_question(newItems_wiki_question)

In [ ]:
from scipy import sparse

In [ ]:
bm25_vectors_question = []
corpus_len_question = len(newItems_wiki_question)
for item in tqdm(newItems_wiki_question):
    doc_len = len(item['question'])
    bm25_vectors_question.append(
        bm25(doc_len, corpus_len_question, item['unigram_question'], textCounter_question)
    )

bm25_vectors = []
corpus_len = len(newItems_wiki)
for item in tqdm(newItems_wiki):
    doc_len = len(item['text'])
    bm25_vectors.append(
        bm25(doc_len, corpus_len, item['unigram_text'], textCounter)
    )

  0%|          | 0/21457 [00:00<?, ?it/s]

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
bm25_vectors_question = sparse.csr_matrix(bm25_vectors_question)
bm25_vectors = sparse.csr_matrix(bm25_vectors)

question_bm25_vectors = bm25_vectors[20857:]
corpus_bm25_vectors = bm25_vectors[:20857]
question_bm25_vectors_question = bm25_vectors_question[20857:]
corpus_bm25_vectors_question = bm25_vectors_question[:20857]

In [ ]:
questions_file = open('e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json')
questions = json.load(questions_file)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")

qa_model = pipeline("question-answering", model=model,tokenizer=tokenizer,device=0)

#You can replace "nguyenvulebinh/vi-mrc-large" by "hogger32/xlmRoberta-for-VietnameseQA"

4

In [ ]:
for i in tqdm(range(600)):    
    query_vector = question_bm25_vectors_question[i]
    query_vector = np.reshape(query_vector, (1,-1))
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_bm25_vectors_question)
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:10]
    result=None
    index_result_score=None
    score=[]
    doc=[]
    question_result_list=[]
    question_result_list_score=[]
    result_list=[]
    result_list_score=[]
    for _id in idx:
        # result = newItems_wiki_question[_id]['question']
        # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        # result_ans = result['answer']
        # result = data['data'][_id]['text']
        # score = sim_maxtrix[_id]
        # questions['data'][i]['doc_score'] = score
        # questions['data'][i]['question_retrieval'] = data['data'][_id]['question']
        result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        question_result_list.append(result['answer'])
        question_result_list_score.append(result['score'])
        score.append(sim_maxtrix[_id])
        doc.append(data['data'][_id]['question'])
    # questions['data'][i]['question_answer'] = doc
    # questions['data'][i]['doc_question_score'] = score
    # questions['data'][i]['question_result'] = result_list
    # questions['data'][i]['question_result_score'] = result_list_score
    index_question_score = np.array(question_result_list_score).argsort()[-1]
    if  data['data'][index_question_score]['category']=="FALSE_LONG_ANSWER":
        # score=[]
        # doc=[]
        
        query_vector = question_bm25_vectors[i]
        query_vector = np.reshape(query_vector, (1,-1))
        sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_bm25_vectors)
        sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
        idx = (-sim_maxtrix).argsort()[:10]
        for _id in idx:                   
            # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
            # result = data['data'][_id]['text']
            # result_ans = result['answer']
            # questions['data'][i]['doc_score'] = sim_maxtrix[_id]
            # questions_file['data'][i]['ans_score'] = result['score']
            result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
            result_list.append(result['answer'])
            result_list_score.append(result['score'])
            score.append(sim_maxtrix[_id])
            doc.append(data['data'][_id]['text'])
        index_result_score = np.array(result_list_score).argsort()[-1]


    # if question_result_list_score[index_question_score] > result_list_score[index_result_score]:
    #     result = question_result_list[index_question_score]
    # else:
    #     result = question_result_list[index_result_score]

    # questions['data'][i]['ans_score'] = result['score']
    # result = result['answer']
    question_result = question_result_list[index_question_score]
    result = question_result
    if result_list != []:  
      if question_result_list_score[index_question_score] < result_list_score[index_result_score]:
        result = result_list[index_result_score]
    
    if is_date_format(result):
        result = to_date_format(result)
    elif has_numbers(result):
        result = to_number_format(result)
    else:
        result = to_wiki_format(result)
    # questions_file['data'][i]['result'] = result
    
    # if is_date_format(question_result):
    #     question_result = to_date_format(question_result)
    # elif has_numbers(question_result):
    #     question_result = to_number_format(question_result)
    # else:
    #     question_result = to_wiki_format(question_result)

    questions['data'][i]['answer'] = result
    # questions['data'][i]['question_answer'] = question_result

    # questions['data'][i]['answer']=doc
    # questions['data'][i]['doc_score'] = score
    # questions['data'][i]['result'] = result_list
    # questions['data'][i]['result_score'] = result_list_score


  0%|          | 0/600 [00:00<?, ?it/s]

In [ ]:
json_object = json.dumps(questions, indent=4, ensure_ascii=False)
with open("submission.json", "w", encoding='utf-8') as result:
    result.write(json_object)